In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
from xiao_utils import f

### 单纯考虑车型的特征（140行）
### 该类型特征一定要分下述这两个版本分别做！
#### 算出两个版本：(OC_这一前缀表示Only Class，即单纯考虑车型的整体情况，不考虑历史各月的不同情况)
    + OC_features_offline.csv —— 统计时不带2017十月的，用于线下验证
    + OC_features_online.csv 统计时带上2017十月的，用于线上提交结果
    
### 该类型特征的 column name 都以 OC_ 开头，意为Only Class

1. 该车型总的记录条数
2. 该车型销售的月数（起售至今几个月了）
3. 该车型销量总数
4. 该车型月均销量，即上一条除以上上条
4. 该车型各种数据的直接的describe（）
    1. 数值型特征的mean, min, max, 中位数, 25%, 75%
1. 车型各项参数的众数mode（依据并非表中出现次数，而是取本车型下销量最多的当前参数作为这一众数）



In [2]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)


In [3]:
# 咱这里所谓的众数 【OC_mode_XXX】
modes = df[['class_id','brand_id']].groupby('class_id').first().reset_index()
for col_name in df.drop(['class_id','sale_date','sale_quantity','brand_id'], axis=1).columns.values:
    gd = df.groupby(['class_id',col_name]).sum().sort_values(ascending=False, by=['sale_quantity']).reset_index()
    modes['OC_mode_' + col_name] = gd.groupby('class_id').first().reset_index()[col_name]

modes

,class_id,brand_id,OC_mode_compartment,OC_mode_type_id,OC_mode_level_id,OC_mode_department_id,OC_mode_TR,OC_mode_gearbox_type,OC_mode_displacement,OC_mode_if_charging,...,OC_mode_engine_torque,OC_mode_car_length,OC_mode_car_width,OC_mode_car_height,OC_mode_total_quality,OC_mode_equipment_quality,OC_mode_rated_passenger,OC_mode_wheelbase,OC_mode_front_track,OC_mode_rear_track
0,103507,831,2,1,3.0,3,6,AT,1.6,L,...,150.7,4270,1780,1630,1830,1325,5,2590,1557,1570
1,124140,783,2,1,1.0,1,6,AT,1.8,T,...,230.0,4650,1850,1695,2075,1665,5,2700,1565,1565
2,125403,761,2,3,2.0,2,6,AT,2.4,L,...,225.0,5213,1878,1772,2470,1840,7,3088,1593,1601
3,136916,106,3,2,2.0,5,4,AT,2.0,L,...,190.0,4825,1825,1480,2000,1465,5,2775,1575,1560
4,169673,682,2,1,2.0,1,6,MT,1.5,T,...,215.0,4715,1830,1780,2095,1570,7,2760,1547,1550
5,175962,750,3,2,1.0,4,5,MT,1.6,L,...,155.0,4473,1706,1469,1600,1120,5,2603,1460,1500
6,178529,98,1,4,1.0,1,5,MT,1.0,L,...,85.0,3730,1510,1860,1575,985,7,2500,1290,1290
7,186250,761,2,2,3.0,2,6,AT,1.5,L,...,146.0,4370,1809,1461,1715,1280,5,2662,1540,1562
8,194201,831,2,2,1.0,3,6,AT,1.4,L,...,132.4,4240,1750,1505,1570,1145,5,2600,1524,1529
9,194450,836,2,1,1.0,3,7,DCT,1.6,T,...,265.0,4475,1850,1655,2090,1541,5,2670,1620,1631


In [4]:
# 下面所提到的ndf，为new df的简称，意为依据指定的特征构造出来的新数据集
# 车型各种信息的统计量
# 【注意】这里要先drop掉sale_quantity，因为实际测试时不可能有这个字段的统计量的！！
# 【扯淡！】这里的所谓sale_quantity，在进行统计时，意义其实是历史上的销量，是可以存在的！！因为最后测试集也是直接拿这个ndf过去join，实际上是取历史销量的信息！是可以有的！
# ndf = df.drop('sale_quantity', axis=1).groupby('class_id').describe()
ndf = df[['class_id',  'price','price_level','sale_quantity','displacement',
          'power','cylinder_number','engine_torque','car_length',
          'car_height','car_width','total_quality','equipment_quality','wheelbase',
          'front_track','rear_track']].groupby('class_id').describe()

# 车型销量总数
ndf['sale_quantity_total'] = df.groupby('class_id')['sale_quantity'].sum()


In [6]:
# 统计每个月有多少个车型在售
selling_cls_in_month = df[['sale_date', 'class_id']].groupby(['sale_date', 'class_id']).size().unstack()
# 这个基础上count一下，就是每个车型一共有多少个月有售
OC_sale_month_cnt = selling_cls_in_month.count()

# 该车型月均销售量
# OC_month_sale_avg = ndf['sale_quantity_total'] / OC_sale_month_cnt


In [7]:
# 上面这几个结果join起来
oc = pd.merge(modes, ndf, how='left', left_on='class_id', right_index=True)
oc = pd.merge(oc, pd.DataFrame(OC_sale_month_cnt, columns=['OC_sale_month_cnt']), how='left', left_on='class_id', right_index=True)
# 该车型月均销售量
oc['OC_month_sale_avg'] = oc[('sale_quantity_total','')] / oc['OC_sale_month_cnt']

oc

C:\Users\xiao\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,class_id,brand_id,OC_mode_compartment,OC_mode_type_id,OC_mode_level_id,OC_mode_department_id,OC_mode_TR,OC_mode_gearbox_type,OC_mode_displacement,OC_mode_if_charging,...,"(wheelbase, mean)","(wheelbase, std)","(wheelbase, min)","(wheelbase, 25%)","(wheelbase, 50%)","(wheelbase, 75%)","(wheelbase, max)","(sale_quantity_total, )",OC_sale_month_cnt,OC_month_sale_avg
0,103507,831,2,1,3.0,3,6,AT,1.6,L,...,2590.000000,0.000000,2590.0,2590.0,2590.0,2590.0,2590.0,17085,32,533.906250
1,124140,783,2,1,1.0,1,6,AT,1.8,T,...,2700.000000,0.000000,2700.0,2700.0,2700.0,2700.0,2700.0,11237,42,267.547619
2,125403,761,2,3,2.0,2,6,AT,2.4,L,...,3088.000000,0.000000,3088.0,3088.0,3088.0,3088.0,3088.0,12280,70,175.428571
3,136916,106,3,2,2.0,5,4,AT,2.0,L,...,2775.000000,0.000000,2775.0,2775.0,2775.0,2775.0,2775.0,21847,70,312.100000
4,169673,682,2,1,2.0,1,6,MT,1.5,T,...,2760.000000,0.000000,2760.0,2760.0,2760.0,2760.0,2760.0,581,5,116.200000
5,175962,750,3,2,1.0,4,5,MT,1.6,L,...,2603.000000,0.000000,2603.0,2603.0,2603.0,2603.0,2603.0,20644,59,349.898305
6,178529,98,1,4,1.0,1,5,MT,1.0,L,...,2557.046980,105.092573,2500.0,2500.0,2500.0,2500.0,2750.0,41205,70,588.642857
7,186250,761,2,2,3.0,2,6,AT,1.5,L,...,2662.000000,0.000000,2662.0,2662.0,2662.0,2662.0,2662.0,2702,21,128.666667
8,194201,831,2,2,1.0,3,6,AT,1.4,L,...,2600.000000,0.000000,2600.0,2600.0,2600.0,2600.0,2600.0,617,2,308.500000
9,194450,836,2,1,1.0,3,7,DCT,1.6,T,...,2646.904762,19.818106,2630.0,2630.0,2630.0,2670.0,2670.0,7588,69,109.971014


In [8]:
# 对类别类型的列做 onehot 编码
oc_before_oh = oc
oc = pd.get_dummies(oc) # 先将object类型的列给哑编码喽
oc = pd.get_dummies(oc, columns=['brand_id','OC_mode_type_id','OC_mode_level_id','OC_mode_department_id','OC_mode_driven_type_id',
                                 'OC_mode_fuel_type_id','OC_mode_newenergy_type_id','OC_mode_if_MPV_id',
                                 'OC_mode_if_luxurious_id']) 

In [9]:
oc.dtypes

class_id                           int64
OC_mode_compartment                int64
OC_mode_displacement             float64
OC_mode_price_level                int64
OC_mode_price                    float64
OC_mode_emission_standards_id      int64
OC_mode_power                    float64
OC_mode_cylinder_number            int64
OC_mode_engine_torque            float64
OC_mode_car_length                 int64
OC_mode_car_width                  int64
OC_mode_car_height                 int64
OC_mode_total_quality              int64
OC_mode_equipment_quality          int64
OC_mode_wheelbase                  int64
OC_mode_front_track                int64
OC_mode_rear_track                 int64
(car_height, count)              float64
(car_height, mean)               float64
(car_height, std)                float64
(car_height, min)                float64
(car_height, 25%)                float64
(car_height, 50%)                float64
(car_height, 75%)                float64
(car_height, max

In [10]:
# 存盘
oc.to_csv("../../data/features/OC_features_online.csv",index=False)

In [12]:
# # 对品牌id做one-hot编码
# ndf_before_oh = ndf
# # df = pd.get_dummies(df, columns=['brand_id','type_id','department_id','TR','gearbox_type',
# #                                     'if_charging','driven_type_id','fuel_type_id',
# #                                     'newenergy_type_id','emission_standards_id',
# #                                     'if_MPV_id','if_luxurious_id','rated_passenger'])
# ndf = pd.get_dummies(ndf, columns=['brand'])

# 上面是online的（带201710的）版本
# 下面弄offline的（不带201710的）版本

In [2]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)


下面是为了在本地用10月数据做验证，所以需要先对数据集做切分，并依据咱指定的规则做特征。  

步骤：
1. 把10月的销量统计出来，作为验证集保存住（只留class_id、sale_date、sale_quantity）；
2. 把10月的从df里删去，后面就还是对df做操作就行。

In [3]:
# 只保留2017年10月以前的（不包括10月）
df = df[df['sale_date'] < pd.to_datetime('201710', format='%Y%m')]

In [4]:
# 咱这里所谓的众数 【OC_mode_XXX】
modes = df[['class_id','brand_id']].groupby('class_id').first().reset_index()
for col_name in df.drop(['class_id','sale_date','sale_quantity','brand_id'], axis=1).columns.values:
    gd = df.groupby(['class_id',col_name]).sum().sort_values(ascending=False, by=['sale_quantity']).reset_index()
    modes['OC_mode_' + col_name] = gd.groupby('class_id').first().reset_index()[col_name]

modes

,class_id,brand_id,OC_mode_compartment,OC_mode_type_id,OC_mode_level_id,OC_mode_department_id,OC_mode_TR,OC_mode_gearbox_type,OC_mode_displacement,OC_mode_if_charging,...,OC_mode_engine_torque,OC_mode_car_length,OC_mode_car_width,OC_mode_car_height,OC_mode_total_quality,OC_mode_equipment_quality,OC_mode_rated_passenger,OC_mode_wheelbase,OC_mode_front_track,OC_mode_rear_track
0,103507,831,2,1,3.0,3,6,AT,1.6,L,...,150.7,4270,1780,1630,1830,1325,5,2590,1557,1570
1,124140,783,2,1,1.0,1,6,AT,1.8,T,...,230.0,4650,1850,1695,2075,1665,5,2700,1565,1565
2,125403,761,2,3,2.0,2,6,AT,2.4,L,...,225.0,5213,1878,1772,2470,1840,7,3088,1593,1601
3,136916,106,3,2,2.0,5,4,AT,2.0,L,...,190.0,4825,1825,1480,2000,1465,5,2775,1575,1560
4,169673,682,2,1,2.0,1,6,MT,1.5,T,...,215.0,4715,1830,1780,2095,1570,7,2760,1547,1550
5,175962,750,3,2,1.0,4,5,MT,1.6,L,...,155.0,4473,1706,1469,1600,1120,5,2603,1460,1500
6,178529,98,1,4,1.0,1,5,MT,1.0,L,...,85.0,3730,1510,1860,1575,985,7,2500,1290,1290
7,186250,761,2,2,3.0,2,6,AT,1.5,L,...,146.0,4370,1809,1461,1715,1280,5,2662,1540,1562
8,194201,831,2,2,1.0,3,6,AT,1.4,L,...,132.4,4240,1750,1505,1570,1145,5,2600,1524,1529
9,194450,836,2,1,1.0,3,7,DCT,1.6,T,...,265.0,4475,1850,1655,2090,1541,5,2670,1620,1631


In [5]:
# 下面所提到的ndf，为new df的简称，意为依据指定的特征构造出来的新数据集
# 车型各种信息的统计量
# 【注意】这里要先drop掉sale_quantity，因为实际测试时不可能有这个字段的统计量的！！
# 【扯淡！】这里的所谓sale_quantity，在进行统计时，意义其实是历史上的销量，是可以存在的！！因为最后测试集也是直接拿这个ndf过去join，实际上是取历史销量的信息！是可以有的！
# ndf = df.drop('sale_quantity', axis=1).groupby('class_id').describe()
ndf = df[['class_id',  'price','price_level','sale_quantity','displacement',
          'power','cylinder_number','engine_torque','car_length',
          'car_height','car_width','total_quality','equipment_quality','wheelbase',
          'front_track','rear_track']].groupby('class_id').describe()

# 车型销量总数
ndf['sale_quantity_total'] = df.groupby('class_id')['sale_quantity'].sum()


In [6]:
# 统计每个月有多少个车型在售
selling_cls_in_month = df[['sale_date', 'class_id']].groupby(['sale_date', 'class_id']).size().unstack()
# 这个基础上count一下，就是每个车型一共有多少个月有售
OC_sale_month_cnt = selling_cls_in_month.count()

# 该车型月均销售量
# OC_month_sale_avg = ndf['sale_quantity_total'] / OC_sale_month_cnt


In [7]:
# 上面这几个结果join起来
oc = pd.merge(modes, ndf, how='left', left_on='class_id', right_index=True)
oc = pd.merge(oc, pd.DataFrame(OC_sale_month_cnt, columns=['OC_sale_month_cnt']), how='left', left_on='class_id', right_index=True)
# 该车型月均销售量
oc['OC_month_sale_avg'] = oc[('sale_quantity_total','')] / oc['OC_sale_month_cnt']

oc

C:\Users\xiao\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,class_id,brand_id,OC_mode_compartment,OC_mode_type_id,OC_mode_level_id,OC_mode_department_id,OC_mode_TR,OC_mode_gearbox_type,OC_mode_displacement,OC_mode_if_charging,...,"(wheelbase, mean)","(wheelbase, std)","(wheelbase, min)","(wheelbase, 25%)","(wheelbase, 50%)","(wheelbase, 75%)","(wheelbase, max)","(sale_quantity_total, )",OC_sale_month_cnt,OC_month_sale_avg
0,103507,831,2,1,3.0,3,6,AT,1.6,L,...,2590.000000,0.000000,2590.0,2590.0,2590.0,2590.0,2590.0,16905,31,545.322581
1,124140,783,2,1,1.0,1,6,AT,1.8,T,...,2700.000000,0.000000,2700.0,2700.0,2700.0,2700.0,2700.0,10974,41,267.658537
2,125403,761,2,3,2.0,2,6,AT,2.4,L,...,3088.000000,0.000000,3088.0,3088.0,3088.0,3088.0,3088.0,12124,69,175.710145
3,136916,106,3,2,2.0,5,4,AT,2.0,L,...,2775.000000,0.000000,2775.0,2775.0,2775.0,2775.0,2775.0,21685,69,314.275362
4,169673,682,2,1,2.0,1,6,MT,1.5,T,...,2760.000000,0.000000,2760.0,2760.0,2760.0,2760.0,2760.0,429,4,107.250000
5,175962,750,3,2,1.0,4,5,MT,1.6,L,...,2603.000000,0.000000,2603.0,2603.0,2603.0,2603.0,2603.0,20407,58,351.844828
6,178529,98,1,4,1.0,1,5,MT,1.0,L,...,2557.239057,105.217536,2500.0,2500.0,2500.0,2500.0,2750.0,41057,69,595.028986
7,186250,761,2,2,3.0,2,6,AT,1.5,L,...,2662.000000,0.000000,2662.0,2662.0,2662.0,2662.0,2662.0,2619,20,130.950000
8,194201,831,2,2,1.0,3,6,AT,1.4,L,...,2600.000000,NaN,2600.0,2600.0,2600.0,2600.0,2600.0,239,1,239.000000
9,194450,836,2,1,1.0,3,7,DCT,1.6,T,...,2646.766467,19.796235,2630.0,2630.0,2630.0,2670.0,2670.0,7317,68,107.602941


In [8]:
# 对类别类型的列做 onehot 编码
oc_before_oh = oc
oc = pd.get_dummies(oc) # 先将object类型的列给哑编码喽
oc = pd.get_dummies(oc, columns=['brand_id','OC_mode_type_id','OC_mode_level_id','OC_mode_department_id','OC_mode_driven_type_id',
                                 'OC_mode_fuel_type_id','OC_mode_newenergy_type_id','OC_mode_if_MPV_id',
                                 'OC_mode_if_luxurious_id']) 

In [9]:
oc.dtypes

class_id                           int64
OC_mode_compartment                int64
OC_mode_displacement             float64
OC_mode_price_level                int64
OC_mode_price                    float64
OC_mode_emission_standards_id      int64
OC_mode_power                    float64
OC_mode_cylinder_number            int64
OC_mode_engine_torque            float64
OC_mode_car_length                 int64
OC_mode_car_width                  int64
OC_mode_car_height                 int64
OC_mode_total_quality              int64
OC_mode_equipment_quality          int64
OC_mode_wheelbase                  int64
OC_mode_front_track                int64
OC_mode_rear_track                 int64
(car_height, count)              float64
(car_height, mean)               float64
(car_height, std)                float64
(car_height, min)                float64
(car_height, 25%)                float64
(car_height, 50%)                float64
(car_height, 75%)                float64
(car_height, max

In [10]:
# 存盘
oc.to_csv("../../data/features/OC_features_offline.csv",index=False)